In [112]:
#May 27th, 2021
#n重振り子を解きたいJulia（色々改良していく枠）

using LinearAlgebra,DifferentialEquations,Plots,StaticArrays

#parameters
n  = 2     # 個
g  = 9.8   # m/s²
Δl = 0.0   # m
Δm = 0.0   # kg
#functionで用いる行列の大きさも含めて宣言しておく．
A = zeros(MMatrix{n,n})
B = zeros(MMatrix{n,n})
U = SVector{n}(-ones(n))
L = SVector{n}(@. 1.0 + (1:n)*Δl )
M = SVector{n}(@. 1.0 + (1:n)*Δm )
p = (;n, g, Δl, Δm, A, B, U, L, M)

#2021-5-27 modified
# L = SVector{n}(@. 1.0 + (1:n)*Δl )
# この表記は次の書き方の省略・効率化 → dot-syntax
#
# L = []
# for i=1:n
#     lᵢ = 1.0 + i*Δl
#     push!(L,lᵢ)
# end
#
# push!() をあらわに書くのを避けること
    
function Npendulum!(du, u, p, t)
    
#2021-5-27 modified
# Mᵢᵢ = sum(M[j] for j=i:n)
# 上記のsum()は下記の処理を行う 
# Mᵢᵢ = 0.0
# for j=i:n
#     Mᵢᵢ += M[j]
# end

#係数行列Aの初期化
    #対角成分
        for i=1:n
            Mᵢᵢ = sum(M[j] for j=i:n)
            A[i,i] = L[i]*L[i]*Mᵢᵢ
        end    
    #非対角成分
        for i=1:n
            for j=(i+1):n
                Mᵢⱼ = sum(M[k] for k=j:n)
                A[i,j] = L[i]*L[j]*cos(u[i]-u[j])*Mᵢⱼ
                A[j,i] = A[i,j]
            end
        end    
#係数行列Bの初期化
    #対角成分
        for i=1:n
            Mᵢᵢ = sum(M[j] for j=i:n)
            B[i,i] = g*L[i]*sin(u[i])*Mᵢᵢ
        end
    #非対角成分
        for i=1:n
            for j=(i+1):n
                Mᵢⱼ = sum(M[k] for k=j:n)
                B[i,j] = L[i]*L[j]*u[n+j]^2*sin(u[i]-u[j])*Mᵢⱼ #2021-5-27 + -> * 修正 
                B[j,i] = L[j]*L[i]*u[n+i]^2*sin(u[j]-u[i])*Mᵢⱼ #2021-5-27 + -> * 修正
            end
        end
    F = A\B*U
    #運動方程式
    for i=1:n
        du[i] = u[n+i]  #dθᵢ=ωᵢ
        du[n+i] = F[i]  #dωᵢ=Fᵢ
    end
end

# Plots setups & initial conditions #
    Sol = []
    Plt = []

    NumofConditions = 1
    θ = zeros(SVector{n})
    ω = zeros(SVector{n})
    t₀ = 0.0
    T = 20.0
    Ly = 2.0*pi
    dcolor = 0.1
###############

#initial conditions : u₀[i]=θᵢ,u₀[n+i]=ωᵢ

@time for ii=1:NumofConditions
    θ = [pi/2.0 for i=1:n]
    ω = [0.0 for i=1:n]
    u₀ = [θ ; ω]
    Tinterval = (t₀,T)

    prob2 = ODEProblem(Npendulum!,u₀,Tinterval,p)
    sol = solve(prob2, Tsit5(), reltol=1e-8, abstol=1e-8)
    
    push!(Sol, sol)

    plt = plot(Sol[ii], lw=3,
               xlim = (t₀, T),ylim = (-Ly,Ly),
               title="n-pendulum: θ1₀=θ2₀=π/2 [rad],ω1₀=ω2₀=0 [rad/s]",
               linestyle = [:solid :solid :dot :dot],
               lc = [RGB(0.0,0.0,dcolor*8) RGB(dcolor*8,0.0,0.0) RGB(0.0,0.0,dcolor*8) RGB(dcolor*8,0.0,0.0)],
               label = ["θ₁" "θ₂" "ω₁" "ω₂"]
        )
    push!(Plt, plt)
end

xlength = 800
ylength = 500

#plot(Plt...; size=(xlength,ylength))

  3.260456 seconds (3.79 M allocations: 206.492 MiB, 3.70% gc time, 95.20% compilation time)


500